# Camada Bronze:
---

## Tabela Geolocalização:

### Objetivo:

Entender a padronização e comportamento dos dados para criar a view geolocalizacao_bronze a partir da camada RAW.

### Fonte:

`workspace.default.geolocalizacao`

### Destino:

`workspace.olist_bronze.geolocalizacao_bronze`

In [0]:
-- Garantindo a existência do SCHEMA para armazenamento:

CREATE SCHEMA IF NOT EXISTS workspace.olist_bronze;

In [0]:
-- Verificando a existência da tabela:
SELECT * FROM workspace.default.geolocalizacao;

In [0]:
-- Verificando quantas linhas tem na tabela:
SELECT COUNT(*) AS LINHAS 
FROM workspace.default.geolocalizacao;

Exitem mais de 1 milhão de registros pois um cliente pode possuir mais de uma localização.

In [0]:
-- Verificando os tipos de dados:
DESC workspace.default.geolocalizacao;

Os tipos de dados estão corretamente definidos, com exceção da coluna `geolocation_zip_code_prefix`, que está tipificada como `BIGINT`. Considerando que o maior código possui quatro algarismos, esse tipo é desnecessariamente grande e resulta em uso ineficiente de memória. Dessa forma, a coluna pode ser convertida para `INT`, mantendo a semântica dos dados e reduzindo o consumo de memória.

In [0]:
-- Verificando missing values em cada coluna:
SELECT
      SUM(CASE WHEN geolocation_zip_code_prefix IS NULL THEN 1 ELSE 0 END) AS geolocation_zip_code_prefix_null,
      SUM(CASE WHEN geolocation_lat IS NULL THEN 1 ELSE 0 END) AS geolocation_lat_null,
      SUM(CASE WHEN geolocation_lng IS NULL THEN 1 ELSE 0 END) AS geolocation_lng_null,
      SUM(CASE WHEN geolocation_city IS NULL THEN 1 ELSE 0 END) AS geolocation_city_null,
      SUM(CASE WHEN geolocation_state IS NULL THEN 1 ELSE 0 END) AS geolocation_state_null
FROM workspace.default.geolocalizacao;

Não há valores nulos em nenhuma coluna.

In [0]:
-- Verificando valores inválidos em cada coluna:
SELECT 
      SUM(CASE WHEN geolocation_zip_code_prefix IS NOT NULL AND NOT geolocation_zip_code_prefix RLIKE '^[0-9]+$' THEN 1 ELSE 0 END) AS geolocation_zip_code_prefix_invalid,

      SUM(CASE WHEN geolocation_lat IS NOT NULL AND CAST(geolocation_lat AS DOUBLE) IS NULL THEN 1 ELSE 0 END) AS geolocation_lat_invalid,

      SUM(CASE WHEN geolocation_lng IS NOT NULL AND CAST(geolocation_lng AS DOUBLE) IS NULL THEN 1 ELSE 0 END) AS geolocation_lng_invalid,

      SUM(CASE WHEN geolocation_city IS NOT NULL AND NOT geolocation_city RLIKE "^[a-z0-9 '-]+$" THEN 1 ELSE 0 END) AS geolocation_city_invalid,

      SUM(CASE WHEN geolocation_state IS NOT NULL AND NOT geolocation_state RLIKE '^[A-Z]+$' THEN 1 ELSE 0 END) AS geolocation_state_invalid

FROM workspace.default.geolocalizacao;

Apenas a coluna `geolocation_city_invalid` apresenta valores inválidos, será necessário observar quais são esse valores para um melhor entendimento.

In [0]:
SELECT 
      geolocation_city
FROM workspace.default.geolocalizacao
WHERE geolocation_city NOT RLIKE "^[a-z0-9 '-]+$";

Os valores cadastrados divergem do padrão por apresentarem nomes de cidades com acentuação, tornando necessário o tratamento desses dados nas camadas subsequentes.

In [0]:
CREATE OR REPLACE VIEW workspace.olist_bronze.geolocalizacao_bronze AS
SELECT *

FROM workspace.default.geolocalizacao;